In [ ]:
import numpy as np
import xarray as xr
import shapely

import rasterio as rio
import rasterio.mask as riomask

from tools.landsat_utils import easy_raster, normalize_band as normalize
from tools.utils import create_polydict_ptsarr, xy2ll, directory_spider

import matplotlib.pyplot as plt
from tools.plot_tools import plot_latlon_lines
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [ ]:
import plotSettings
plotSettings.paper(1.4)
COLS = plotSettings.COLS

In [ ]:
%matploblib tk

In [ ]:
# general data directory
data_dir = '/'

In [ ]:
# bounding box locations for context image
bb_filename = os.path.join(data_dir, 'GIS/BoundingBoxes2.shp')
print('loading mask')
bbdict, bbpts = create_polydict_ptsarr(bb_filename)
mask = bbdict[0]

doline_img = 'data/LandSat8-9_Color/LC08_L1GT_128111_20190124_20201016_02_T2'

# worldview location and file paths
WV_dir = 'data/WorldView_Maxar/'
WV_files = directory_spider(os.path.join(data_dir, WV_dir), path_pattern='_PAN', file_pattern='TIF')
WV_files = [f for f in WV_files if '.aux' not in f and '._' not in f]
# file paths for each image in 2015 and 2020
before_file = [f for f in WV_files if 'EPSG3031' in f][0]
after_file = [f for f in WV_files if 'EPSG3031' in f][1]

In [ ]:
# a temporary class to store WorldView/Maxar raster data and info
class Raster:
    def __init__(self, filepath, mask=None):
        mosaic = rio.open(filepath)
        self.mosaic = mosaic
        self.raster = (mosaic.read(1)).astype(float)
        self.raster[self.raster == 0] = np.nan
        self.crs = mosaic.crs
        self.width = mosaic.width
        self.height = mosaic.height
        self.transform = mosaic.transform
        self.bounds = mosaic.bounds
        self.mask = mask
        if mask is not None:
            masked, _ = riomask.mask(mosaic, [mask], crop=True, filled=False)
            self.masked = masked[0]

In [ ]:
# EPSG3031 bounds for these particular fractures at these dates
NW_frac_before = [1843200.565, 686102.868, 1844143.501, 686966.597] # 2015
NW_frac_after =  [1843655.383, 686269.213, 1844598.3199999998, 687132.943] #2020  

SE_frac_before = [1843323.622, 685394.147, 1844022.170, 686092.694] # 2015
SE_frac_after =  [1843675.265, 685442.577, 1844373.812, 686141.124] # 2020

In [ ]:
SAVE = False
save_name = 'figure7'
save_dir = 'figs'
save_type = '.svg'

########################################################################################################
# create figure
########################################################################################################
plt.close('all')
fig = plt.figure(num=1,clear=1, figsize=[13.52533333,  7.79733333])
P = gridspec.GridSpec(1,2,figure=fig, width_ratios=[1,1.4])
gs0 = gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=P[0])
imax = fig.add_subplot(gs0[0])

gs1 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=P[1], hspace=0.01)
ax = np.array([[fig.add_subplot(gs1[0]), fig.add_subplot(gs1[1])],
               [fig.add_subplot(gs1[2]), fig.add_subplot(gs1[3])]])

########################################################################################################
# plot LS8 doline image
########################################################################################################
LS = easy_raster(os.path.join(data_dir, doline_img),
                 bands=['B2','B3','B4','B8'],
                 mask=mask)
ps = LS.pan_sharpen('brovey')
red,green,blue = 'B4','B3','B2'
R,G,B = ps[red],  ps[green], ps[blue]
MIN = [0,0,0]
MAX = [np.quantile(ps[b], 0.98) for b in [red, green, blue]]
shape = R.shape
RGB = np.empty((shape[0], shape[1], 3),dtype=int)
RGB[:,:,0] = np.clip(255*normalize(R, MIN[0], MAX[0]), 0, 255)
RGB[:,:,1] = np.clip(255*normalize(G, MIN[1], MAX[1]), 0, 255)
RGB[:,:,2] = np.clip(255*normalize(B, MIN[2], MAX[2]), 0, 255)

left, bottom, right, top = mask.bounds
imax.imshow(RGB, extent=(left, right, bottom, top), rasterized=True)

scalebar = AnchoredSizeBar(imax.transData,
                           1e3, '1 km', 'lower left', 
                           pad=1,
                           frameon=False,
                           size_vertical=100,
                          )

imax.add_artist(scalebar)
left, bottom, right, top = mask.bounds
lon, lat = xy2ll(mask.boundary.xy[0], mask.boundary.xy[1])
lon_range = [np.min(lon), np.max(lon)]
lat_range = [np.min(lat), np.max(lat)]
imax = plot_latlon_lines(imax, lat_range=lat_range, lon_range=lon_range,
                         extents=(left, right, bottom, top),
                         lon_text=True)

imax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[],
         xlim=(left, right), ylim=(bottom, top))

########################################################################################################
# NW fracture WorldView/Maxar image
########################################################################################################
coords =((NW_frac_before[0], NW_frac_before[1]),
         (NW_frac_before[0], NW_frac_before[3]),
         (NW_frac_before[2], NW_frac_before[3]),
         (NW_frac_before[2], NW_frac_before[1]),
         (NW_frac_before[0], NW_frac_before[1])
        )
qm = shapely.geometry.Polygon(coords)

before = Raster(before_file, mask=qm)
extent = (qm.bounds[0], qm.bounds[2], qm.bounds[1], qm.bounds[3])
ax[0,0].imshow(before.masked, cmap='gray', extent=extent, rasterized=True, aspect='equal')

coords =((NW_frac_after[0], NW_frac_after[1]),
         (NW_frac_after[0], NW_frac_after[3]),
         (NW_frac_after[2], NW_frac_after[3]),
         (NW_frac_after[2], NW_frac_after[1]),
         (NW_frac_after[0], NW_frac_after[1])
        )
qm = shapely.geometry.Polygon(coords)

after = Raster(after_file, mask=qm)
extent = (qm.bounds[0], qm.bounds[2], qm.bounds[1], qm.bounds[3])
ax[0,1].imshow(after.masked, cmap='gray', extent=extent, rasterized=True, aspect='equal')
imax.plot([NW_frac_after[0], NW_frac_after[0], NW_frac_after[2], NW_frac_after[2], NW_frac_after[0]],
          [NW_frac_after[1], NW_frac_after[3], NW_frac_after[3], NW_frac_after[1], NW_frac_after[1]],
          c=COLS[0]
         )
ax[0,1].set(xlim=(NW_frac_after[0], NW_frac_after[2]),
            ylim=(NW_frac_after[1], NW_frac_after[3]))
scalebar = AnchoredSizeBar(ax[0,0].transData,
                           250, '250 m', 'lower left', 
                           pad=1,
                           frameon=False,
                           size_vertical=25,
                          )

ax[0,0].add_artist(scalebar)

########################################################################################################
# NW fracture WorldView/Maxar image
########################################################################################################
coords =((SE_frac_before[0], SE_frac_before[1]),
         (SE_frac_before[0], SE_frac_before[3]),
         (SE_frac_before[2], SE_frac_before[3]),
         (SE_frac_before[2], SE_frac_before[1]),
         (SE_frac_before[0], SE_frac_before[1])
        )
qm = shapely.geometry.Polygon(coords)

before = Raster(before_file, mask=qm)
extent = (qm.bounds[0], qm.bounds[2], qm.bounds[1], qm.bounds[3])
ax[1,0].imshow(before.masked, cmap='gray', extent=extent, rasterized=True, aspect='equal')

scalebar = AnchoredSizeBar(ax[1,0].transData,
                           250, '250 m', 'lower left', 
                           pad=1,
                           frameon=False,
                           size_vertical=25,
                          )

ax[1,0].add_artist(scalebar)

# qm = quick_mask(SE_frac_after)
coords =((SE_frac_after[0], SE_frac_after[1]),
         (SE_frac_after[0], SE_frac_after[3]),
         (SE_frac_after[2], SE_frac_after[3]),
         (SE_frac_after[2], SE_frac_after[1]),
         (SE_frac_after[0], SE_frac_after[1])
        )
qm = shapely.geometry.Polygon(coords)


after = Raster(after_file, mask=qm)
extent = (qm.bounds[0], qm.bounds[2], qm.bounds[1], qm.bounds[3])
ax[1,1].imshow(after.masked, cmap='gray', extent=extent, rasterized=True, aspect='equal')
imax.plot([SE_frac_after[0], SE_frac_after[0], SE_frac_after[2], SE_frac_after[2], SE_frac_after[0]],
          [SE_frac_after[1], SE_frac_after[3], SE_frac_after[3], SE_frac_after[1], SE_frac_after[1]],
          c=COLS[1]
         )

[a.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[]) for a in ax.flatten()]
[a.set_linewidth(1) for _,a in imax.spines.items()]
[[a.set_color(COLS[0]) for _, a in _a.spines.items()] for _a in [ax[0,0], ax[0,1]]]
[[a.set_color(COLS[1]) for _, a in _a.spines.items()] for _a in [ax[1,0], ax[1,1]]]
 
if SAVE:
    plt.savefig(os.path.join(save_dir, save_name + fig_type), dpi=600.)